In [2]:
print("The dataset is loaded successfully 😄.")

The dataset is loaded successfully 😄.


In [ ]:
def ingest_session(session, date, user_id, session_id, frame, client):
    messages = []
    if frame == "zep":
        for idx, msg in enumerate(session):
            print(
                f"\033[90m[{frame}]\033[0m 📝 User \033[1;94m{user_id}\033[0m 💬 Session \033[1;94m{session_id}\033[0m: [\033[93m{idx + 1}/{len(session)}\033[0m] Ingesting message: \033[1m{msg['role']}\033[0m - \033[96m{msg['content'][:50]}...\033[0m at \033[92m{date.isoformat()}\033[0m"
            )
            client.memory.add(
                session_id=session_id,
                messages=[
                    Message(
                        role=msg["role"],
                        role_type=msg["role"],
                        content=msg["content"][:8000],
                        created_at=date.isoformat(),
                    )
                ],
            )
    elif frame == "mem0-local" or frame == "mem0-api":
        for idx, msg in enumerate(session):
            messages.append({"role": msg["role"], "content": msg["content"][:8000]})
            print(
                f"\033[90m[{frame}]\033[0m 📝 Session \033[1;94m{session_id}\033[0m: [\033[93m{idx + 1}/{len(session)}\033[0m] Reading message: \033[1m{msg['role']}\033[0m - \033[96m{msg['content'][:50]}...\033[0m at \033[92m{date.isoformat()}\033[0m"
            )
        if frame == "mem0-local":
            client.add(
                messages=messages, user_id=user_id, run_id=session_id, timestamp=date.isoformat()
            )
        elif frame == "mem0-api":
            client.add(
                messages=messages,
                user_id=user_id,
                session_id=session_id,
                timestamp=int(date.timestamp()),
                version="v2",
            )
        print(
            f"\033[90m[{frame}]\033[0m ✅ Session \033[1;94m{session_id}\033[0m: Ingested \033[93m{len(messages)}\033[0m messages at \033[92m{date.isoformat()}\033[0m"
        )
    elif frame == "memobase":
        for idx, msg in enumerate(session):
            messages.append(
                {
                    "role": msg["role"],
                    "content": msg["content"][:8000],
                    "created_at": date.isoformat(),
                }
            )
            print(
                f"\033[90m[{frame}]\033[0m 📝 User \033[1;94m{user_id}\033[0m 💬 Session \033[1;94m{session_id}\033[0m: [\033[93m{idx + 1}/{len(session)}\033[0m] Ingesting message: \033[1m{msg['role']}\033[0m - \033[96m{msg['content'][:50]}...\033[0m at \033[92m{date.isoformat()}\033[0m"
            )

        real_uid = string_to_uuid(user_id)
        user = client.get_user(real_uid)
        memobase_add_memory(user, messages)
        user.flash(sync=True)
        print(
            f"\033[90m[{frame}]\033[0m ✅ Session \033[1;94m{session_id}\033[0m: Ingested \033[93m{len(messages)}\033[0m messages at \033[92m{date.isoformat()}\033[0m"
        )
    elif frame == "memos-local" or frame == "memos-api":
        for _idx, msg in enumerate(session):
            messages.append(
                {
                    "role": msg["role"],
                    "content": msg["content"][:8000],
                    "chat_time": date.isoformat(),
                }
            )
        client.add(messages=messages, user_id=user_id)
        print(
            f"\033[90m[{frame}]\033[0m ✅ Session \033[1;94m{session_id}\033[0m: Ingested \033[93m{len(messages)}\033[0m messages at \033[92m{date.isoformat()}\033[0m"
        )
        client.mem_reorganizer_wait()


def ingest_conv(lme_df, version, conv_idx, frame):
    conversation = lme_df.iloc[conv_idx]

    sessions = conversation["haystack_sessions"]
    dates = conversation["haystack_dates"]

    user_id = "lme_exper_user_" + str(conv_idx)

    print("\n" + "=" * 80)
    print(f"🔄 \033[1;36mINGESTING CONVERSATION {conv_idx}\033[0m".center(80))
    print("=" * 80)

    if frame == "zep":
        client = zep_client()
        print("🔌 \033[1mUsing \033[94mZep client\033[0m \033[1mfor ingestion...\033[0m")
        # Delete existing user and session if they exist
        client.user.delete(user_id)
        print(f"🗑️  Deleted existing user \033[93m{user_id}\033[0m from Zep memory...")
        # Add user to Zep memory
        client.user.add(user_id=user_id)
        print(f"➕ Added user \033[93m{user_id}\033[0m to Zep memory...")
    elif frame == "mem0-local":
        client = mem0_client(mode="local")
        print("🔌 \033[1mUsing \033[94mMem0 Local client\033[0m \033[1mfor ingestion...\033[0m")
        # Delete existing memories for the user
        client.delete_all(user_id=user_id)
        print(f"🗑️  Deleted existing memories for user \033[93m{user_id}\033[0m...")
    elif frame == "mem0-api":
        client = mem0_client(mode="api")
        print("🔌 \033[1mUsing \033[94mMem0 API client\033[0m \033[1mfor ingestion...\033[0m")
        # Delete existing memories for the user
        client.delete_all(user_id=user_id)
        print(f"🗑️  Deleted existing memories for user \033[93m{user_id}\033[0m...")
    elif frame == "memos-local":
        client = memos_client(
            mode="local",
            db_name=f"lme_{frame}-{version}",
            user_id=user_id,
            top_k=20,
            mem_cube_path=f"results/lme/{frame}-{version}/storages/{user_id}",
            mem_cube_config_path="configs/mu_mem_cube_config.json",
            mem_os_config_path="configs/mos_memos_config.json",
            addorsearch="add",
        )
        print("🔌 \033[1mUsing \033[94mMemos Local client\033[0m \033[1mfor ingestion...\033[0m")
    elif frame == "memos-api":
        client = memos_client(mode="api")
    elif frame == "memobase":
        client = memobase_client()
        print("🔌 \033[1mUsing \033[94mMemobase client\033[0m \033[1mfor ingestion...\033[0m")
        client.delete_user(string_to_uuid(user_id))
        print(f"🗑️  Deleted existing user \033[93m{user_id}\033[0m from Memobase memory...")

    for idx, session in enumerate(sessions):
        session_id = user_id + "_lme_exper_session_" + str(idx)
        if frame == "zep":
            client.memory.add_session(
                user_id=user_id,
                session_id=session_id,
            )
            print(
                f"➕ Added session \033[93m{session_id}\033[0m for user \033[93m{user_id}\033[0m to Zep memory..."
            )

        if len(session) == 0:
            print(f"\033[93m⚠️  Skipping empty session {idx} in conversation {conv_idx}\033[0m")
            continue

        date = dates[idx] + " UTC"
        date_format = "%Y/%m/%d (%a) %H:%M UTC"
        date_string = datetime.strptime(date, date_format).replace(tzinfo=timezone.utc)

        try:
            ingest_session(session, date_string, user_id, session_id, frame, client)
        except Exception as e:
            print(f"\033[91m❌ Error ingesting session: {e}\033[0m")

    if frame == "memos-local":
        client.mem_reorganizer_off()
    print("=" * 80)

: 

In [ ]:
import json

# 正确读取JSON Lines格式文件的方法
# 这种格式每行包含一个独立的JSON对象

def read_jsonl_file(file_path):
    """
    读取JSON Lines格式文件
    
    Parameters
    ----------
    file_path : str
        文件路径
        
    Returns
    -------
    list
        包含所有JSON对象的列表
    """
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if line:  # 跳过空行
                try:
                    data.append(json.loads(line))
                except json.JSONDecodeError as e:
                    print(f"第 {line_num} 行JSON解析错误: {e}")
                    print(f"错误内容: {line[:100]}...")
    return data

# 读取数据
try:
    data = read_jsonl_file("../data_agent_multi_turn.json")
    print(f"✅ 成功读取 {len(data)} 条记录")
    
    # 显示第一条记录的基本信息
    if data:
        first_record = data[0]
        print(f"\n📋 第一条记录信息:")
        print(f"   ID: {first_record.get('id', 'N/A')}")
        print(f"   问题长度: {len(first_record.get('question', ''))}")
        print(f"   功能数量: {len(first_record.get('function', []))}")
        
except FileNotFoundError:
    print("❌ 文件未找到，请检查文件路径")
except Exception as e:
    print(f"❌ 读取文件时出错: {e}")


✅ 成功读取 30 条记录

📋 第一条记录信息:
   ID: agent_multi_turn_0
   问题长度: 127
   功能数量: 8


: 

In [ ]:
data[0]

{'id': 'agent_multi_turn_0',
 'question': '你叫Grace，你想给Frank发一条消息，内容是：“Are you free tomorrow to have lunch with me?”。你会对agent做出反应，但不会主动提供任何未被询问的信息，如果需要删除短信，一般删除离现在时间最久的短信。',
 'initial_config': {'BaseApi': {'wifi': False, 'logged_in': True}},
 'path': [],
 'function': [{'name': 'turn_on_wifi',
   'description': '开启WiFi连接。',
   'parameters': {'type': 'dict', 'required': [], 'properties': {}}},
  {'name': 'login_device',
   'description': '登录设备。',
   'parameters': {'type': 'dict', 'required': [], 'properties': {}}},
  {'name': 'send_message',
   'description': '将一条消息从一个用户发送给另一个用户。',
   'parameters': {'type': 'dict',
    'required': ['sender_name', 'receiver_name', 'message'],
    'properties': {'sender_name': {'type': 'string',
      'description': '发送消息的用户姓名。'},
     'receiver_name': {'type': 'string', 'description': '接收消息的用户姓名。'},
     'message': {'type': 'string', 'description': '要发送的消息内容。'}}}},
  {'name': 'view_messages_between_users',
   'description': '获取特定用户发送给另一个用户的所有消息。',
   'para

: 

: 

In [ ]:
# 其他读取JSON Lines的方法

# 方法1：使用列表推导式（简洁版）
def read_jsonl_simple(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line.strip()) for line in f if line.strip()]

# 方法2：使用pandas（如果安装了pandas）
def read_jsonl_with_pandas(file_path):
    try:
        import pandas as pd
        return pd.read_json(file_path, lines=True).to_dict('records')
    except ImportError:
        print("pandas未安装，无法使用此方法")
        return None

# 方法3：批处理读取（适用于大文件）
def read_jsonl_batch(file_path, batch_size=1000):
    """分批读取大型JSON Lines文件"""
    with open(file_path, 'r', encoding='utf-8') as f:
        batch = []
        for line in f:
            line = line.strip()
            if line:
                batch.append(json.loads(line))
                if len(batch) >= batch_size:
                    yield batch
                    batch = []
        if batch:  # 返回最后一批
            yield batch

# 演示不同方法
print("🔄 使用不同方法读取JSON Lines文件:")
print("\n1️⃣ 简洁版读取:")
try:
    data_simple = read_jsonl_simple("../data_agent_multi_turn.json")
    print(f"   读取到 {len(data_simple)} 条记录")
except Exception as e:
    print(f"   错误: {e}")

print("\n2️⃣ Pandas读取:")
data_pandas = read_jsonl_with_pandas("../data_agent_multi_turn.json")
if data_pandas:
    print(f"   读取到 {len(data_pandas)} 条记录")

print("\n3️⃣ 批处理读取 (前2批):")
try:
    batch_count = 0
    for batch in read_jsonl_batch("../data_agent_multi_turn.json", batch_size=10):
        batch_count += 1
        print(f"   第 {batch_count} 批: {len(batch)} 条记录")
        if batch_count >= 2:  # 只显示前2批
            break
except Exception as e:
    print(f"   错误: {e}")


: 

In [ ]:
import json 

: 

In [ ]:
import json
from datetime import datetime

with open("../memory_benchmark/longmemeval/longmemeval_s.json", 'r') as f:
    data = json.load(f)

print(data[0])

{'question_id': 'e47becba', 'question_type': 'single-session-user', 'question': 'What degree did I graduate with?', 'answer': 'Business Administration', 'question_date': '2023/05/30 (Tue) 23:40', 'haystack_dates': ['2023/05/20 (Sat) 02:21', '2023/05/20 (Sat) 02:57', '2023/05/20 (Sat) 03:51', '2023/05/20 (Sat) 05:50', '2023/05/20 (Sat) 10:58', '2023/05/20 (Sat) 15:03', '2023/05/20 (Sat) 20:47', '2023/05/21 (Sun) 03:24', '2023/05/21 (Sun) 09:27', '2023/05/21 (Sun) 13:12', '2023/05/21 (Sun) 17:10', '2023/05/21 (Sun) 22:48', '2023/05/22 (Mon) 04:25', '2023/05/22 (Mon) 09:23', '2023/05/22 (Mon) 11:21', '2023/05/22 (Mon) 11:37', '2023/05/22 (Mon) 14:27', '2023/05/22 (Mon) 14:37', '2023/05/22 (Mon) 18:02', '2023/05/23 (Tue) 01:23', '2023/05/23 (Tue) 03:04', '2023/05/23 (Tue) 09:46', '2023/05/23 (Tue) 13:25', '2023/05/23 (Tue) 21:24', '2023/05/24 (Wed) 13:03', '2023/05/25 (Thu) 03:27', '2023/05/25 (Thu) 03:43', '2023/05/25 (Thu) 09:47', '2023/05/25 (Thu) 22:20', '2023/05/26 (Fri) 04:41', '2023

: 

In [ ]:
dt = datetime.strptime(data[0]["question_date"], '%Y/%m/%d (%a) %H:%M')

: 

In [ ]:
dt.strftime("%Y-%m-%d (%a) %H:%M")

'2023-05-30 (Tue) 23:40'

: 

In [ ]:
import functools

@functools.wraps
def decorator(func):
    def wrapper(*args, **kwargs):
        print("before")
        print(args)
        print(kwargs)
        result = func(*args, **kwargs)
        print("after")
        return result
    return wrapper

@decorator
def f(x, y=1):
    return x + y



: 

In [ ]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("before")
        print(args)
        print(kwargs)
        result = func(*args, **kwargs)
        print("after")
        return result
    return wrapper

@decorator
def f(x, y=1):
    return x + y

: 

In [ ]:
f(1, 2)

before
(1, 2)
{}
after


3

: 

: 

: 

In [ ]:
dt.isoformat()

'2023-05-30T23:40:00'

: 

In [ ]:
size = 0
for session in data[0]["haystack_sessions"]:
    size += len(session)
size

550

: 

In [ ]:
list(data[0].keys())

['question_id',
 'question_type',
 'question',
 'answer',
 'question_date',
 'haystack_dates',
 'haystack_session_ids',
 'haystack_sessions',
 'answer_session_ids']

: 

In [ ]:
from litellm import create_pretrained_tokenizer

: 

In [ ]:
from litellm.types.utils import SelectTokenizerResponse
from litellm import token_counter

: 

In [ ]:
from transformers import AutoTokenizer  
tokenizer = AutoTokenizer.from_pretrained("/disk3/dengxinle/pretrained_models/Qwen2.5-32B-Instruct")

: 

In [ ]:
tokenizer.encode("3432423423")

[18, 19, 18, 17, 19, 17, 18, 19, 17, 18]

: 

In [ ]:
tokenizer = create_pretrained_tokenizer("/disk3/dengxinle/pretrained_models/Qwen2.5-32B-Instruct")

23:45:43 - LiteLLM:ERROR: utils.py:1787 - Error creating pretrained tokenizer: Tokenizer.from_pretrained() got an unexpected keyword argument 'auth_token'. Defaulting to version without 'auth_token'.
/data2/dengxinle/anaconda3/envs/AMEM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/disk3/dengxinle/pretrained_models/Qwen2.5-32B-Instruct'. Use `repo_type` argument if needed.

: 

In [ ]:
from datetime import datetime

: 

In [ ]:
"s12" > "s2"

False

: 

In [ ]:
from memories.datasets.longmemeval import LongMemEval
from memories.layers.amem import AMEMConfig, AMEMLayer
from monkey_patch import (
    PatchSpec, 
    MonkeyPatcher, 
    make_attr_patch, 
)
from token_monitor import CostStateManager, token_monitor

: 

In [ ]:
DATASET_MAPPING = {
    "LongMemEval": LongMemEval,
}

MEMORY_LAYER_MAPPING = {
    "amem": AMEMLayer,
}

MEMORY_CONFIG_MAPPING = {
    "amem": AMEMConfig,
}

: 

In [ ]:
class A:
    def __repr__(self): return "A-repr"
x = A()
print(x)

A-repr


: 

In [ ]:
dataset = LongMemEval.read_raw_data("../memory_benchmark/longmemeval/longmemeval_s.json")

: 

In [ ]:
dataset

LongMemEval Metadata
────────────────────
  avg_message_per_session    : 10.339586299304916
  avg_question_per_trajectory: 1.0
  avg_session_per_trajectory : 47.764
  codebase_url               : 'https://github.com/xiaowu0162/LongMemEval'
  name                       : 'LongMemEval'
  paper                      : 'LongMemEval: Benchmarking Chat Assistants on Long-Term Interactive Memory'
  question_type_stats        :
          knowledge-update         : 78
          multi-session            : 133
          single-session-assistant : 56
          single-session-preference: 30
          single-session-user      : 70
          temporal-reasoning       : 133
  size                       : 500
  total_messages             : 246930
  total_questions            : 500
  total_sessions             : 23882

: 

In [ ]:
s = '2023/05/30 (Tue) 22:20'
dt = datetime.strptime(s, '%Y/%m/%d (%a) %H:%M')

: 

In [ ]:
dt

datetime.datetime(2023, 5, 30, 22, 20)

: 

In [ ]:
list(data[0].keys())

['question_id',
 'question_type',
 'question',
 'answer',
 'question_date',
 'haystack_dates',
 'haystack_session_ids',
 'haystack_sessions',
 'answer_session_ids']

: 

In [ ]:
data[0]["answer"]

'Business Administration'

: 

In [ ]:
data[0]["answer_session_ids"]

['answer_280352e9']

: 

In [ ]:
data[0]["haystack_session_ids"]

['sharegpt_yywfIrx_0',
 '85a1be56_1',
 'sharegpt_Jcy1CVN_0',
 'sharegpt_Cr2tc1f_0',
 'sharegpt_DGTCD7D_0',
 'f6859b48_2',
 '52c34859_1',
 'ultrachat_231069',
 'sharegpt_qRdLQvN_7',
 'ultrachat_359984',
 'sharegpt_8dJs7Ai_0',
 'c3cd646c_1',
 '4fd76316',
 '7045db85_1',
 'ultrachat_422229',
 'ultrachat_215809',
 '41abc171_4',
 '5dac7cc2_1',
 'sharegpt_rnG0ZuV_0',
 '3418b277_3',
 'sharegpt_UnjngE7_65',
 'aae4411b_2',
 '2ae4f277_2',
 'sharegpt_jVow2vn_0',
 'ultrachat_45576',
 'ultrachat_530213',
 '47f00a5f',
 '978c0e70',
 'ultrachat_336116',
 'sharegpt_T1EiHWI_13',
 '3b3a77d9',
 'a041cdd4_3',
 'sharegpt_1L5GXZJ_0',
 '0e9551fc_1',
 'ultrachat_574556',
 'sharegpt_gTknhGs_0',
 'd414cac5_4',
 'ultrachat_214101',
 'ultrachat_286075',
 'ultrachat_57566',
 'sharegpt_PdnvIns_0',
 'ultrachat_514031',
 'ec6ca9ef',
 'd94b721b',
 'sharegpt_QZMeA7V_17',
 'sharegpt_BvwQVCO_0',
 'e348269f_1',
 '0586f435_3',
 'ultrachat_458322',
 '02bd2b90_3',
 'e27891d3_2',
 'ultrachat_486410',
 'answer_280352e9',
 '84503

: 

In [ ]:
data[0]["haystack_sessions"]

[[{'role': 'user',
   'content': "The farmer needs to transport a fox, a chicken, and some grain across a river using a boat. The fox cannot be left alone with the chicken, and the chicken cannot be left alone with the grain. The boat can only hold one item at a time, and the river is too dangerous to cross multiple times. Can you help the farmer transport all three items across the river without any of them getting eaten? Remember, strategic thinking and planning are key to solving this puzzle. If you're stuck, try thinking about how you would solve the puzzle yourself, and use that as a starting point. Be careful not to leave the chicken alone with the fox, or the chicken and the grain alone together, as this will result in a failed solution. Good luck!"},
  {'role': 'assistant',
   'content': 'To solve this puzzle, the farmer can follow these steps:\n\n1. First, the farmer should take the chicken across the river using the boat.\n2. Next, the farmer should go back to the original si

: 

In [ ]:
data[0]["question_type"]

'single-session-user'

: 

In [ ]:
data[2]["question_date"]

'2023/05/30 (Tue) 20:42'

: 

In [ ]:
data[3]["haystack_dates"][-1]

'2023/05/30 (Tue) 22:20'

: 

: 

In [ ]:
list(data[0].keys())

['question_id',
 'question_type',
 'question',
 'answer',
 'question_date',
 'haystack_dates',
 'haystack_session_ids',
 'haystack_sessions',
 'answer_session_ids']

: 

In [ ]:
data[0]["answer_session_ids"]

['answer_280352e9']

: 

In [ ]:
from datetime import datetime
import re

FMT = "%Y/%m/%d %H:%M"

def parse_ts(ts: str) -> datetime:
    clean = re.sub(r"\s*\([^)]+\)\s*", " ", ts).strip()
    return datetime.strptime(clean, FMT)

def diff_between(ts1: str, ts2: str):
    d1, d2 = parse_ts(ts1), parse_ts(ts2)
    delta = d2 - d1
    return 

: 

In [ ]:
delay = None
idx = 0
for i, sample in enumerate(data):
    start, end = parse_ts(sample["haystack_dates"][0]), parse_ts(sample["haystack_dates"][-1])
    if delay is None:
        delay = end - start
    elif end - start > delay:
        delay = end - start
        idx = i
        
idx
        

414

: 

In [ ]:
for timestamp in data[414]["haystack_dates"]:
    print(timestamp)

2022/08/21 (Sun) 11:08
2022/08/22 (Mon) 21:56
2022/08/23 (Tue) 07:21
2022/08/24 (Wed) 23:13
2022/08/25 (Thu) 00:13
2022/08/26 (Fri) 14:11
2022/08/27 (Sat) 04:22
2022/08/28 (Sun) 22:48
2022/08/28 (Sun) 03:10
2022/08/28 (Sun) 03:13
2022/09/01 (Thu) 15:26
2022/09/03 (Sat) 01:21
2022/09/06 (Tue) 09:05
2022/09/07 (Wed) 11:13
2022/09/08 (Thu) 16:16
2022/09/10 (Sat) 04:58
2022/09/10 (Sat) 05:19
2022/09/15 (Thu) 11:08
2022/09/16 (Fri) 23:50
2022/09/16 (Fri) 17:22
2022/09/18 (Sun) 02:44
2022/09/21 (Wed) 00:56
2022/09/28 (Wed) 18:55
2022/10/27 (Thu) 12:43
2022/11/09 (Wed) 21:26
2022/11/17 (Thu) 11:45
2022/12/16 (Fri) 12:23
2022/12/18 (Sun) 15:51
2022/12/21 (Wed) 02:54
2022/12/24 (Sat) 04:49
2022/12/27 (Tue) 23:03
2022/12/30 (Fri) 17:42
2023/01/24 (Tue) 01:39
2023/02/03 (Fri) 04:40
2023/03/02 (Thu) 07:47
2023/03/07 (Tue) 12:53
2023/03/15 (Wed) 06:16
2023/04/19 (Wed) 16:42
2023/04/20 (Thu) 05:37
2023/04/26 (Wed) 08:37
2023/05/27 (Sat) 06:48
2023/05/30 (Tue) 02:02
2023/05/30 (Tue) 08:58
2023/05/31 

: 

In [ ]:
print(len(data[414]["haystack_dates"]))

48


: 

In [ ]:
for conversation in data:
    sessions = conversation["haystack_sessions"]
    dates = conversation["haystack_dates"]

500


: 

In [ ]:
import json

with open("../shared_contexts_32k.jsonl", 'r') as f:
    data = [json.loads(line) for line in f]

value = list(data[0].values())
print(len(value[0]))
for item in value[0]:
    print(item)

183
{'role': 'system', 'content': "Current user persona: Name: Kanoa Manu   \nGender Identity: Male  \nRacial Identity: Pacific Islander  \n\nKanoa Manu is a 32-year-old software engineer with a passion for music and technology. Born to a family that cherished traditional Pacific Islander music, he has always been inspired to blend these rich cultural sounds with modern digital melodies. Currently, he is deeply involved in a project where he's experimenting with MIDI files, aiming to create a fusion of electronic and traditional music. Kanoa spends his weekends tinkering with various software tools and musical instruments, always looking for innovative ways to showcase his heritage through sound. His ultimate goal is to develop an app that helps musicians easily incorporate traditional Pacific sounds into contemporary music."}
{'role': 'user', 'content': "User: Hi there! I've recently been diving deeper into my passion for music and technology. It's been quite a journey since I started

: 

In [ ]:
import json 
with open("../SocioVerse/user_pool_X.json", 'r') as f:
    data = json.load(f)

: 

In [ ]:
data[1]

{'user_id': 237237507,
 'screen_name': 'Nancy نانسي Handabile',
 'user_at_name': 'arushapot',
 'overlap_score': 0.200818927591234,
 'RELIGIOUS': 'No religion',
 'EMPLOYMENT': 'Working now',
 'MARITAL': 'Never married',
 'RACE': 'Black Alone',
 'IDEOLOGY': 'Liberal',
 'INCOME': 'Middle Income',
 'AREA': 'Cities',
 'PARTY': 'Independent',
 'AGE': 'Youth (18-35 years old)',
 'GENDER': 'Female'}

: 

In [ ]:
import pandas as pd 
data = pd.read_csv("../Amazon-M2/sessions_train.csv")

: 

In [ ]:
data

,prev_items,next_item,locale
0,['B09W9FND7K' 'B09JSPLN1M'],B09M7GY217,DE
1,['B076THCGSG' 'B007MO8IME' 'B08MF65MLV' 'B001B...,B001B4THSA,DE
2,['B0B1LGXWDS' 'B00AZYORS2' 'B0B1LGXWDS' 'B00AZ...,B0767DTG2Q,DE
3,['B09XMTWDVT' 'B0B4MZZ8MB' 'B0B7HZ2GWX' 'B09XM...,B0B4R9NN4B,DE
4,['B09Y5CSL3T' 'B09Y5DPTXN' 'B09FKD61R8'],B0BGVBKWGZ,DE
...,...,...,...
3606244,['B086CYFSKW' 'B0874F9859' 'B086CYFSKW'],B07B5TYD76,IT
3606245,['B09NRZKZ7V' 'B08WJTPV93'],B08L1P4C3D,IT
3606246,['B085JFX7MP' 'B085JGHW8R'],B01MPWVD44,IT
3606247,['B00B0UING2' 'B00B0UING2'],B00D3HYEZ4,IT


: 

: 

In [ ]:
from litellm import token_counter

: 

In [ ]:
token_counter(model="gpt-4o-mini", messages=[
    {"role": "user", "content": "Hello, world!"}, 
    {"role": "assistant", "content": "Hello, world!"}, 
],
count_response_tokens=True,
)

16

: 

In [ ]:
from litellm import token_counter as litellm_token_counter
from litellm.types.llms.openai import (
    ChatCompletionNamedToolChoiceParam,
    ChatCompletionToolParam,
)

: 

In [ ]:
from litellm import token_counter as litellm_token_counter
from litellm.types.utils import SelectTokenizerResponse
from litellm.types.llms.openai import ChatCompletionNamedToolChoiceParam, ChatCompletionToolParam

: 

In [ ]:
from collections import deque

: 

In [ ]:
q = deque(maxlen=10)

: 

In [ ]:
q.maxlen

10

: 

In [ ]:
import functools
import logging
import time
from typing import Any, Callable, Dict, List, Optional, Union
from collections import defaultdict
import json
from collections import deque
from threading import RLock
from litellm import token_counter as litellm_token_counter
import asyncio
from litellm.types.utils import SelectTokenizerResponse
from litellm.types.llms.openai import ChatCompletionNamedToolChoiceParam, ChatCompletionToolParam


class CostState: 
    """Cost state for a specific LLM model."""

    def __init__(
        self, 
        input_tokens: int = 0,
        output_tokens: int = 0,
        total_time: float = 0.0, 
        window_size: int = 100_000,
        total_count: int = 0,
        histories: Optional[List[Dict[str, Any]]] = None,
    ) -> "CostState":
        self.input_tokens = input_tokens
        self.output_tokens = output_tokens
        self.total_time = total_time
        self.total_count = total_count
        self.histories = (
            deque(maxlen=window_size) 
            if histories is None else 
            deque(histories, maxlen=window_size)
        )
        self._lock = RLock() 

    @property 
    def total_tokens(self) -> int:
        """Compute the total number of tokens."""
        with self._lock:
            return self.input_tokens + self.output_tokens

    @property
    def average_input_tokens(self) -> float:
        """Compute the average number of input tokens per call."""
        with self._lock:
            return self.input_tokens / max(self.total_count, 1)

    @property
    def average_output_tokens(self) -> float:
        """Compute the average number of output tokens per call."""
        with self._lock:
            return self.output_tokens / max(self.total_count, 1)
        
    @property
    def average_tokens_per_call(self) -> float:
        """Compute the average number of tokens per call."""
        with self._lock:
            return self.total_tokens / max(self.total_count, 1)

    @property
    def average_time_per_call(self) -> float:
        """Compute the average time per call."""
        with self._lock:
            return self.total_time / max(self.total_count, 1)
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert the cost state to a dictionary."""
        with self._lock:
            return {
                "total_count": self.total_count,
                "total_tokens": self.total_tokens,
                "average_input_tokens": self.average_input_tokens,
                "average_output_tokens": self.average_output_tokens,
                "average_tokens_per_call": self.average_tokens_per_call,
                "average_time_per_call": self.average_time_per_call,
                "histories": list(self.histories),
                "total_time": self.total_time,
                "input_tokens": self.input_tokens,
                "output_tokens": self.output_tokens,
                "window_size": self.histories.maxlen
            }

    def update(
        self, 
        input_tokens: int, 
        output_tokens: int,
        total_time: float,
        histories: List[Dict[str, Any]],
    ) -> None:
        """Update the cost state."""    
        with self._lock:
            self.input_tokens += input_tokens
            self.output_tokens += output_tokens
            self.total_time += total_time
            self.total_count += len(histories)
            self.histories.extend(histories)

    
    def to_json(self) -> str:
        """Convert the cost state to a JSON string."""
        return json.dumps(
            self.to_dict(), 
            indent=4, 
            sort_keys=True,
            ensure_ascii=False, 
        )
    
    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> "CostState":
        """Create a cost state from a dictionary."""
        allowed = [
            "input_tokens", 
            "output_tokens", 
            "total_time", 
            "window_size", 
            "total_count", 
            "histories", 
        ]
        kwargs = {k: data[k] for k in allowed if k in data}
        return cls(**kwargs)
    
    @classmethod
    def from_json(cls, json_str: str) -> "CostState":
        """Create a cost state from a JSON string."""
        return cls.from_dict(json.loads(json_str))

: 

In [ ]:
import torch

: 

In [ ]:
torch.inference_mode

: 

In [ ]:
from datetime import datetime
import time 


: 

In [ ]:
start_time = datetime.now().astimezone()
start_timestamp = start_time.strftime("%Y-%m-%d %H:%M:%S %z")
time.sleep(10)
end_time = datetime.now().astimezone()
end_timestamp = end_time.strftime("%Y-%m-%d %H:%M:%S %z")

: 

In [ ]:
(end_time - start_time).total_seconds() / 60

0.16667276666666667

: 

In [ ]:
now_local.strftime("%Y-%m-%d %H:%M:%S %z")

: 

In [ ]:
date_format = "%Y/%m/%d (%a) %H:%M UTC"
date_string = datetime.strptime(date, date_format).replace(tzinfo=timezone.utc)

: 

In [ ]:
from litellm import create_pretrained_tokenizer, create_tokenizer


: 

In [ ]:
from litellm import encoding

: 

In [ ]:
from litellm import encode

: 

In [ ]:
encode("gpt-4o-mini", "I have an apple.")

[40, 617, 459, 24149, 13]

: 

In [ ]:
encode("gpt-3.5-turbo", "I have an apple.")

[40, 617, 459, 24149, 13]

: 

In [ ]:
from litellm import encode

: 

In [ ]:
create_pretrained_tokenizer

: 

In [ ]:
from collections import deque

: 

In [ ]:
q = deque(
    [
        ["sz", "sds"], 
        ["sds", 'sds', 'ds', 's']
    ]
)

: 

In [ ]:
q.append(["232", '2'])

: 

: 

In [ ]:
def dec(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

: 

In [ ]:
from openai import OpenAI

: 

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key="sk-RbEjV9qrdLGmwgnJdf6lPvqxJso9f95r6BGaGP2eANuVCnci", 
    base_url="https://www.DMXapi.com/v1",
)

def dec(func):
    def wrapper(*args, **kwargs):
        print(1)
        return func(*args, **kwargs)
    return wrapper

client.chat.completions.create = dec(client.chat.completions.create)


: 

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello, how are you?"}]
)

1


: 

In [ ]:
import openai
import time 

: 

In [ ]:
openai.OpenAI.chat.completions.__class__.create = time.sleep 

AttributeError: 'cached_property' object has no attribute 'completions'

: 

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from chromadb.config import Settings

model_name = "/disk3/dengxinle/pretrained_models/modernbert-embed-base"

: 

In [ ]:
client = chromadb.Client(Settings(allow_reset=True))
embedding_function = SentenceTransformerEmbeddingFunction(model_name=model_name)
collection = client.get_or_create_collection(name="memories", embedding_function=embedding_function)

/data2/dengxinle/anaconda3/envs/AMEM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

In [ ]:
collection.add(
    documents=["Hello, world!"],
    metadatas=[{"source": "my_source"}],
    ids=["id1"]
)

: 

In [ ]:
fetched = collection.get(
    ids=["id1"], include=["documents", "metadatas", "embeddings"]
)

: 

In [ ]:
type(fetched["embeddings"])

numpy.ndarray

: 

In [ ]:
import pickle
with open("test.pkl", "wb") as f:
    pickle.dump(fetched["embeddings"][0], f)



: 

In [ ]:
with open("test.pkl", 'rb') as f:
    data = pickle.load(f)

: 

In [ ]:
data

array([ 1.51248137e-02, -8.28075409e-03,  6.99032471e-03,  5.27386554e-04,
        4.31540364e-04, -4.54376265e-02, -3.15571502e-02, -3.91542651e-02,
        2.97269393e-02, -6.83437893e-03,  3.14338841e-02, -1.29521368e-02,
        1.44764418e-02, -6.31215051e-02,  5.35456883e-03,  2.47500092e-02,
        9.29515157e-03, -9.36552789e-03, -7.73581043e-02,  1.97017100e-02,
        2.89467685e-02,  4.51054890e-03,  3.46519519e-03,  1.39686652e-03,
       -4.87552844e-02, -6.89822063e-03, -3.04973926e-02,  7.64956372e-03,
        1.95779074e-02,  1.97067037e-02, -3.25245745e-02, -8.19213688e-02,
        8.29148758e-03,  7.37822279e-02, -2.30970774e-02,  1.63280088e-02,
       -3.00512705e-02, -1.32831158e-02,  4.44476269e-02, -2.72768047e-02,
       -1.48795517e-02,  1.12027675e-02,  4.38164994e-02, -4.25760262e-02,
       -1.18965907e-02, -3.92893255e-02, -3.00750323e-02, -5.88287413e-02,
       -1.78384427e-02, -2.39421446e-02,  5.03755361e-02,  1.85485110e-02,
        3.29874828e-02,  

: 

In [ ]:
response

ChatCompletion(id='chatcmpl-C7KtWFEBAI4YNeZGaCYKeWHB9IGLd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1755864650, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_efad92c60b', usage=CompletionUsage(completion_tokens=31, prompt_tokens=13, total_tokens=44, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

: 

In [ ]:
list(q)

[['sz', 'sds'], ['sds', 'sds', 'ds', 's'], ['232', '2']]

: 

In [ ]:
from memories.layers.amem import AMEMConfig, AMEMLayer

/mnt/dengxinle/anaconda3/envs/AMEM/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


: 

In [ ]:
import os 
os.environ["OPENAI_API_KEY"] = "sk-Y0bpFbtmUr1P2O33587a4a629aDf4aFeB50165C6BcA8A34c"
os.environ["OPENAI_API_BASE"] = "https://api.gptplus5.com/v1"


: 

In [ ]:
for user_id in [
    # "LongMemEval_0", 
    # "LongMemEval_1", 
    # "LongMemEval_2", 
    "LongMemEval_3", 
]:
    config = AMEMConfig(
        user_id="guest", 
        retriever_name_or_path="/mnt/dengxinle/pretrained_models/modernbert-embed-base",
        save_dir=f"./A-MEM/{user_id}"
    )
    layer = AMEMLayer(config)
    layer.load_memory(user_id)
    for key, value in layer.memory_layer.memories.items():
        print(value.content)
        print(
            layer.memory_layer.retriever.collection.get(
                ids=[key], include=["documents", "metadatas", "embeddings"]
            )
        )
    break

Speaker user says: I'm planning a team outing for my engineers and I need some suggestions for fun activities in the city. Do you have any recommendations?
{'ids': ['7a9412b4-c993-461b-8ebc-dba0aa358fa4'], 'embeddings': array([[ 3.81018966e-02, -4.11873907e-02, -2.49658898e-02,
         1.46375112e-02,  4.92573977e-02, -2.46731061e-02,
         3.20314914e-02,  1.82699561e-02,  6.32281369e-03,
        -5.52322567e-02, -3.76366861e-02, -1.73563529e-02,
        -1.29070543e-02, -3.59205827e-02,  9.96430498e-03,
        -4.17103171e-02,  2.06156708e-02,  6.65860280e-05,
        -3.50862592e-02, -5.80827259e-02,  3.23872045e-02,
         1.18792644e-02, -1.31024951e-02,  2.64136512e-02,
        -2.99482550e-02,  2.43536737e-02, -7.38147646e-04,
         3.22689749e-02, -9.60880332e-03,  1.05913272e-02,
         2.82023475e-02, -3.30546461e-02, -3.10491100e-02,
         1.10822665e-02,  8.60823598e-03, -3.73327583e-02,
         2.59808954e-02,  1.11382315e-02,  4.30066958e-02,
         3.58

: 

In [ ]:
for value in layer.memory_layer.memories.values():
    

{'29a6e5b1-bd79-41fa-a261-73d52a5d8b18': <memories.layers.agentic_memory.memory_system.MemoryNote at 0x7d69027c1630>}

: 

: 

: 

: 

In [ ]:
import json 
with open("../data_agent_multi_turn.json", 'r', encoding="utf-8") as f:
    data = json.load(f)

JSONDecodeError: Extra data: line 2 column 1 (char 1962)

: 

: 

: 

In [ ]:
import pandas as pd 
data = pd.read_parquet("../OPeRA/OPeRA_filtered/action/train/train.parquet")
data_group = data.groupby("session_id")

: 

In [ ]:
for i, (session_id, history) in enumerate(data_group):
    if i > 3:
        break 

: 

In [ ]:
history

,session_id,action_id,timestamp,action_type,click_type,semantic_id,target,mouse_position,element_meta,url,window_size,page_meta,simplified_html,rationale,products,input_text
2691,009b6e0c-9aff-41cc-842e-45658133dec6_2025-04-1...,c5fb41b2-8d4a-41af-80c4-bc1a48c435e5,2025-04-11T19:41:05.616Z,click,product_option,product_options.size.button_list.9_women_7_men,"{""tagName"": ""INPUT"", ""className"": ""a-button-in...","{""clientX"": 691, ""clientY"": 545, ""pageX"": 691,...","{""name"": ""product_options"", ""data"": ""{\""Size\""...",https://www.amazon.com/Crocs-Unisex-Womens-Lit...,"{""width"": 1280, ""height"": 665}","{""search_term"": [{""term"": """"}], ""cart_items"": ...","<html><head></head><body><div name=""nav_bar"" r...",,[],None
2692,009b6e0c-9aff-41cc-842e-45658133dec6_2025-04-1...,d66e91ce-dd8b-485b-a34c-813c00e58077,2025-04-11T19:42:30.891Z,click,review,reviews.popover.close,"{""tagName"": ""BUTTON"", ""className"": "" a-button-...","{""clientX"": 788, ""clientY"": 87, ""pageX"": 788, ...","{""name"": """", ""data"": """"}",https://www.amazon.com/Crocs-Unisex-Womens-Lit...,"{""width"": 1280, ""height"": 665}","{""search_term"": [{""term"": """"}], ""cart_items"": ...","<html><head></head><body><div name=""nav_bar"" r...",,[],None
2693,009b6e0c-9aff-41cc-842e-45658133dec6_2025-04-1...,327974da-1a22-481c-8565-e0ede59654f6,2025-04-11T19:42:44.169Z,click,product_option,product_options.color.button_list.59_99_59_99_...,"{""tagName"": ""INPUT"", ""className"": ""a-button-in...","{""clientX"": 650, ""clientY"": 209, ""pageX"": 650,...","{""name"": ""product_options"", ""data"": ""{\""Color\...",https://www.amazon.com/Crocs-Unisex-Womens-Lit...,"{""width"": 1280, ""height"": 665}","{""search_term"": [{""term"": """"}], ""cart_items"": ...","<html><head></head><body><div name=""nav_bar"" r...",,[],None
2694,009b6e0c-9aff-41cc-842e-45658133dec6_2025-04-1...,fe3a52c5-82d3-4421-ab39-2220b38817b7,2025-04-11T19:43:31.289Z,click,nav_bar,nav_bar.account_and_list_button,"{""tagName"": ""DIV"", ""className"": ""nav-div nav-a...","{""clientX"": 948, ""clientY"": 41, ""pageX"": 948, ...","{""name"": """", ""data"": """"}",https://www.amazon.com/Crocs-Unisex-Womens-Lit...,"{""width"": 1280, ""height"": 665}","{""search_term"": [{""term"": """"}], ""cart_items"": ...","<html><head></head><body><div name=""nav_bar"" r...",,[],None
2695,009b6e0c-9aff-41cc-842e-45658133dec6_2025-04-1...,c8953ed8-2fb5-4517-8bc0-774be506f8c9,2025-04-11T19:44:14.384Z,click,nav_bar,nav_bar.account_and_list_button,"{""tagName"": ""DIV"", ""className"": ""nav-div nav-a...","{""clientX"": 942, ""clientY"": 35, ""pageX"": 942, ...","{""name"": """", ""data"": """"}",https://www.amazon.com/Crocs-Unisex-Womens-Lit...,"{""width"": 1280, ""height"": 665}","{""search_term"": [{""term"": """"}], ""cart_items"": ...","<html><head></head><body><div name=""nav_bar"" r...",,[],None
2696,009b6e0c-9aff-41cc-842e-45658133dec6_2025-04-1...,30657e4f-45d1-4cc3-b616-85f70c45a95a,2025-04-11T20:08:21.248Z,click,search,nav_bar.search_input,"{""tagName"": ""INPUT"", ""className"": ""nav-input n...","{""clientX"": 486, ""clientY"": 20, ""pageX"": 486, ...","{""name"": ""search_term"", ""data"": ""{\""term\"":\""\...",https://www.amazon.com/gp/cart/view.html?URL_M...,"{""width"": 1280, ""height"": 665}","{""search_term"": [{""term"": """"}]}",<html><head><title>Amazon.com Shopping Cart</t...,,[],None
2697,009b6e0c-9aff-41cc-842e-45658133dec6_2025-04-1...,23befc75-55d6-40f2-ac4b-741f75bbc12a,2025-04-11T20:08:23.144Z,click,suggested_term,nav_bar.suggested_terms.men_s_mules_clogs,"{""tagName"": ""DIV"", ""className"": ""cards_carouse...","{""clientX"": 429, ""clientY"": 125, ""pageX"": 429,...","{""name"": """", ""data"": """"}",https://www.amazon.com/gp/cart/view.html?URL_M...,"{""width"": 1280, ""height"": 665}","{""search_term"": [{""term"": """"}], ""promotion_ite...",<html><head><title>Amazon.com Shopping Cart</t...,,[],None
2698,009b6e0c-9aff-41cc-842e-45658133dec6_2025-04-1...,9a2e6f

: 

In [ ]:
import json
import ast
from typing import Any, Union, Dict, List

def _maybe_parse_string(value: str) -> Any:
    """
    Try to parse a string as JSON, otherwise as a Python literal. 
    Return the original string if both fail.
    """
    s = value.strip()
    # Quick guard: only attempt when string looks like a container/primitive in JSON/Python
    if not (s.startswith("{") or s.startswith("[") or s.startswith('"') or s.startswith("'")):
        return value
    # Prefer strict JSON first
    try:
        return json.loads(s)
    except Exception:
        pass
    # Fallback to Python literal (handles single quotes, tuples, etc.)
    try:
        return ast.literal_eval(s)
    except Exception:
        return value

def parse_nested_serialized(obj: Any, max_depth: int = 5) -> Any:
    """
    Recursively parse nested serialized structures inside JSON-parsed objects.

    Parameters
    ----------
    obj : Any
        The input object, typically obtained from json.loads, which may contain
        strings that are themselves serialized dict/list structures.
    max_depth : int, default 5
        Maximum recursive parsing depth to avoid infinite recursion on adversarial inputs.

    Returns
    -------
    Any
        The object with nested serialized strings parsed into native Python types
        (dict/list/number/bool/None) wherever possible.
    """
    if max_depth <= 0:
        return obj

    if isinstance(obj, dict):
        return {k: parse_nested_serialized(v, max_depth - 1) for k, v in obj.items()}

    if isinstance(obj, list):
        return [parse_nested_serialized(v, max_depth - 1) for v in obj]

    if isinstance(obj, str):
        parsed = _maybe_parse_string(obj)
        # If parsing changed the value, keep recursing
        if parsed is not obj:
            return parse_nested_serialized(parsed, max_depth - 1)
        return obj

    return obj

: 

In [ ]:
parse_nested_serialized(history.iloc[3, -4])

'<html><head></head><body><div name="nav_bar" role="navigation" aria-label="Primary"><div name="nav_bar.homepage">Homepage</div><form role="search"><select name="nav_bar.search_drop_down_list" title="Search in"></select><input name="nav_bar.search_input" value="" type="text" role="searchbox" aria-label="Search Amazon"><input name="nav_bar.search_button" type="submit" value="Go"></form><div name="nav_bar.account_and_list_button">Account &amp; Lists</div><a name="nav_bar.order_button" href="/gp/css/order-history?ref_=nav_orders_first">Returns &amp; Orders</a><a name="nav_bar.cart_button" aria-label="1 item in cart" href="/gp/cart/view.html?ref_=nav_cart">Cart</a><div name="nav_bar.suggested_terms" role="dialog"></div><a name="nav_bar.menu" role="button" aria-label="Open All Categories Menu" href="javascript: void(0)">Menu</a><div name="nav_bar.stores"><li name="nav_bar.stores.amazon_haul">Amazon Haul</li><li name="nav_bar.stores.same_day_delivery">Same-Day Delivery</li><li name="nav_bar.

: 

In [ ]:
history.iloc[-2, -3]

''

: 

In [ ]:
parse_nested_serialized(history.iloc[3, -3]) == parse_nested_serialized(history.iloc[4, -3])

True

: 

In [ ]:
parse_nested_serialized(history.iloc[7, 11])

{'search_term': [{'term': ''}],
 'cart_items': [{'title': 'SENSYNE 62" Phone Tripod & Selfie Stick, Extendable Cell Phone Tripod Stand with Wireless Remote and Phone Holder, Compatible with iPhone Android Phone, Camera',
   'asin': 'B09TQY66NH',
   'price': '$19.99',
   'url': '/gp/product/B09TQY66NH/ref=ewc_pr_img_1?smid=A1PE0ETKWCIUO0&psc=1',
   'quantity': '1'}],
 'promotion_items': [{'title': 'Unisex LiteRide 360 Clogs',
   'price': '$59.99',
   'url': '/Crocs-Unisex-Womens-LiteRide-Clogs/dp/B0962DM2HH'},
  {'title': 'Women and Men Arch Support Clogs Slip-on Garden Shoes Outdoor Beach Slippers Sandals with Plantar Fasciitis Feet Cushion Insoles',
   'asin': '',
   'price': '$23.39',
   'url': '/VZQ-Support-Slippers-Fasciitis-9-5-10-5/dp/B0C4LC5816/ref=pd_ci_mcx_mh_di_ci_mcx_cn_deals_d_0_title?pd_rd_w=EX0dY&content-id=amzn1.sym.b3c70c26-9d9d-49dd-a6fb-b1195d716cac&pf_rd_p=b3c70c26-9d9d-49dd-a6fb-b1195d716cac&pf_rd_r=P2T3S0S0SXCTXJ6WYYSN&pd_rd_wg=aBhmb&pd_rd_r=54f71b9a-7f8a-4e5e-91fd

: 

In [ ]:
from langchain.chat_models import init_chat_model
import os 
# os.environ["OPENAI_API_KEY"] = "sk-UeZijRSOLATIQ3ztg8gXFpIMCq2AEEugw1kqHHREj2eKoUPc"
# os.environ["OPENAI_API_BASE"] = "https://api.gpts.vin/v1"

os.environ["OPENAI_API_KEY"] = "sk-Y0bpFbtmUr1P2O33587a4a629aDf4aFeB50165C6BcA8A34c"
os.environ["OPENAI_API_BASE"] = "https://api.gptplus5.com/v1"

: 

In [ ]:
?OpenAI

Init signature:
OpenAI(
    *,
    api_key: 'str | None | Callable[[], str]' = None,
    organization: 'str | None' = None,
    project: 'str | None' = None,
    webhook_secret: 'str | None' = None,
    base_url: 'str | httpx.URL | None' = None,
    websocket_base_url: 'str | httpx.URL | None' = None,
    timeout: 'Union[float, Timeout, None, NotGiven]' = NOT_GIVEN,
    max_retries: 'int' = 2,
    default_headers: 'Mapping[str, str] | None' = None,
    default_query: 'Mapping[str, object] | None' = None,
    http_client: 'httpx.Client | None' = None,
    _strict_response_validation: 'bool' = False,
) -> 'None'
Docstring:     
Abstract base class for generic types.

A generic type is typically declared by inheriting from
this class parameterized with one or more type variables.
For example, a generic mapping type might be defined as::

  class Mapping(Generic[KT, VT]):
      def __getitem__(self, key: KT) -> VT:
          ...
      # Etc.

This class can then be used as follows::

  def

: 

: 

: 

: 

: 

: 

: 

In [ ]:
parse_nested_serialized(history.iloc[4, 11])

{'search_term': [{'term': ''}],
 'cart_items': [{'title': 'SENSYNE 62" Phone Tripod & Selfie Stick, Extendable Cell Phone Tripod Stand with Wireless Remote and Phone Holder, Compatible with iPhone Android Phone, Camera',
   'asin': 'B09TQY66NH',
   'price': '$19.99',
   'url': '/gp/product/B09TQY66NH/ref=ewc_pr_img_1?smid=A1PE0ETKWCIUO0&psc=1',
   'quantity': '1'}],
 'product_details': [{'title': 'Crocs Unisex LiteRide 360 Clogs'},
  {'price': '$51.55'},
  {'bullet_list': "Science And Magic: Take A Ride In A Superbly Cushioned New Kind Of Crocs For Men And Women. Inspired By Your Evolving Lifestyle, The Literide Shoe Collection Was Created To Be Worn On Your Terms, At Any Pace. Innovative Comfort: Next Generation 360 Degrees Of Literide Foam Insoles Are Super-Soft, Incredibly Light And Extraordinary Resilient. These Women'S And Men'S Shoes Offer Sink-In Softness For Innovative Comfort. What Size Should I Buy?: These Shoes Offer A Roomy Fit And We Recommend Ordering A Size Down To The N

: 

: 

In [ ]:
from langgraph.store.memory import InMemoryStore

: 

In [ ]:
from pydantic import (
    BaseModel, 
    Field, 
    model_validator,
)

: 

In [ ]:
help(Field)

Help on function Field in module pydantic.fields:

Field(default: 'Any' = PydanticUndefined, *, default_factory: 'Callable[[], Any] | Callable[[dict[str, Any]], Any] | None' = PydanticUndefined, alias: 'str | None' = PydanticUndefined, alias_priority: 'int | None' = PydanticUndefined, validation_alias: 'str | AliasPath | AliasChoices | None' = PydanticUndefined, serialization_alias: 'str | None' = PydanticUndefined, title: 'str | None' = PydanticUndefined, field_title_generator: 'Callable[[str, FieldInfo], str] | None' = PydanticUndefined, description: 'str | None' = PydanticUndefined, examples: 'list[Any] | None' = PydanticUndefined, exclude: 'bool | None' = PydanticUndefined, discriminator: 'str | types.Discriminator | None' = PydanticUndefined, deprecated: 'Deprecated | str | bool | None' = PydanticUndefined, json_schema_extra: 'JsonDict | Callable[[JsonDict], None] | None' = PydanticUndefined, frozen: 'bool | None' = PydanticUndefined, validate_default: 'bool | None' = PydanticUnde

: 

In [1]:
from __future__ import annotations
from memories.layers.base import BaseMemoryLayer 
from langgraph.store.memory import InMemoryStore
from langchain.chat_models import init_chat_model
from langchain_core.language_models import BaseChatModel
from memories.layers.baselines.langmem import create_memory_store_manager

from pydantic import (
    BaseModel, 
    Field, 
    model_validator,
    field_validator,
)
import pickle 
import os
import json
from typing import (
    List, 
    Dict, 
    Any,
    Optional, 
) 

class LangMemConfig(BaseModel):
    """The default configuration for LangMem."""

    user_id: str = Field(..., description="The user id of the memory system.")
    retriever_name_or_path: str = Field(
        default="huggingface:all-MiniLM-L6-v2",
        description="The name or path of the retriever model to use. "
        "The format should be `<provider>:<model_name>` where `<provider>` is one of `huggingface`, `openai`, `ollama`, etc. "
        "and `<model_name>` is the name of the model to use. "
        "For example, `huggingface:all-MiniLM-L6-v2` is the name of the all-MiniLM-L6-v2 model on Hugging Face.",
    )
    retriever_dim: int = Field(
        default=384,
        ge=1, 
        description="The dimension of the retriever model. "
        "The default value is 384, which is the dimension of the all-MiniLM-L6-v2 model on Hugging Face. "
        "If you changes the value of `retriever_name_or_path`, "
        "you need to change the value of `retriever_dim` to the dimension of the new model.",
    )
    llm_model: str = Field(
        default="openai:gpt-4o-mini",
        description="The base backbone model to use. "
        "The format should be `<provider>:<model_name>` where `<provider>` is one of `openai`, `ollama`, etc. "
        "and `<model_name>` is the name of the model to use. "
        "For example, `openai:gpt-4o-mini` is the name of the gpt-4o-mini model on OpenAI.", 
    )

    save_dir: str = Field(default="langmem", description="The directory to save the memory.")

    # You can look up the following parameters in the `create_memory_store_manager` function.
    query_model: str | None = Field(
        default=None, 
        description="The model to use for generating queries. "
        "If not provided, the dialated window trick over the conversation "
        "is used to generate queries and the number of queries is controlled by `query_limit`. "
        "The format should be `<provider>:<model_name>` where `<provider>` is one of `openai`, `ollama`, etc. "
        "and `<model_name>` is the name of the model to use. "
        "For example, `openai:gpt-4o-mini` is the name of the gpt-4o-mini model on OpenAI.", 
    )
    enable_inserts: bool = Field(
        default=True, 
        description="Whether to allow creating new memory entries. "
        "When False, the manager will only update existing memories. Defaults to True.",
    )
    enable_deletes: bool = Field(
        default=True, 
        description="Whether to allow deleting existing memories "
        "that are outdated or contradicted by new information. Defaults to True.", 
    )

    # Before the agent needs to extract valuable information to be memorized
    # it firsts generate a list of queries to retrieve relevant memories.
    # The `query_limit` is the maximum number of related memories to retrieve for each query.
    # When `query_model` is not provided, the dialated window trick over the conversation 
    # is used to generate queries and the number of queries is controlled by `query_limit` (at most `query_limit // 4`).
    query_limit: int = Field(
        default=5,
        ge=1, 
        description="Maximum number of relevant memories to retrieve " 
        "for each conversation. Higher limits provide more context but may slow down processing. "
        "Defaults to 5.",
    )

    @field_validator("retriever_name_or_path", "llm_model", "query_model")
    @classmethod
    def _validate_provider_model(cls, v: Optional[str]) -> Optional[str]:
        if v is None:
            return v
        if ':' not in v:
            raise ValueError("Must be in format '<provider>:<model_name>' (missing ':').")
        provider, model_name = v.split(':', 1)
        if not provider or not model_name:
            raise ValueError("Provider and model name must be non-empty, separated by ':'.")
        return v
    
    @model_validator(mode="after")
    def _validate_save_dir(self) -> LangMemConfig:
        if os.path.isfile(self.save_dir):
            raise AssertionError(f"Provided path ({self.save_dir}) should be a directory, not a file")
        return self 

class LangMemLayer(BaseMemoryLayer):

    layer_type: str = "langmem"

    def __init__(self, config: LangMemConfig) -> None:
        """Create an interface of LangMem. The implemenation is based on the 
        third-party library `langmem`."""
        self._llm_model = init_chat_model(config.llm_model)
        self._query_model = (
            None
            if config.query_model is None
            else init_chat_model(config.query_model)
        )
        self._store = InMemoryStore(
            index={
                "dims": config.retriever_dim, 
                "embed": config.retriever_name_or_path, 
                "fields": ["content"],   # `kind` is ignored as there is only one kind of memory. 
            }
        )
        self.memory_layer = create_memory_store_manager(
            self._llm_model,
            enable_inserts=config.enable_inserts, 
            enable_deletes=config.enable_deletes, 
            query_model=self._query_model,  
            query_limit=config.query_limit, 
            namespace=("memories", config.user_id),  
            store=self._store, 
        )
        self.config = config 

        # Store each memory unit's id 
        self._memory_ids = {}  
    
    @property
    def llm_model(self) -> BaseChatModel:
        """Get the LLM backbone model."""
        return self._llm_model

    def add_message(self, message: Dict[str, str], **kwargs) -> None:
        """Add a message to the memory layer."""
        # See https://langchain-ai.github.io/langmem/background_quickstart/
        # `kwargs` can include some optional parameters, e.g., `max_steps`. 
        final_puts = self.memory_layer.invoke({"messages": [message]}, **kwargs)
        # Some operations update contents of previous memory units. 
        for final_put in final_puts: 
            self._memory_ids[final_put["key"]] = final_put["value"]

    def add_messages(self, messages: List[Dict[str, str]], **kwargs) -> None:
        """Add a list of messages to the memory layer."""    
        message_level = kwargs.pop("message_level", True)
        if message_level not in [True, False]:
            raise TypeError(
                "`message_level` must be a boolean to indicate whether the messages " 
                "are added to the memory layer message by message or as a whole."
            )
        
        if message_level:
            for message in messages: 
                self.add_message(message, **kwargs)
        else:
            final_puts = self.memory_layer.invoke({"messages": messages}, **kwargs)
            for final_put in final_puts: 
                self._memory_ids[final_put["key"]] = final_put["value"]
    
    def retrieve(self, query: str, k: int = 10, **kwargs) -> List[Dict[str, str | Dict[str, Any]]]:
        """Retrieve the memories."""
        memories = self.memory_layer.search(query=query, limit=k, **kwargs)
        outputs = [] 
        for memory in memories:
            memory_dict = memory.dict()
            outputs.append(
                {
                    "content": memory_dict["value"]["content"], 
                    "metadata": {
                        key: value
                        for key, value in memory_dict.items() if key != "value"
                    }
                }
            )
        return outputs  

    def delete(self, memory_id: str) -> bool:
        """Delete the memory."""
        try:
            self.memory_layer.delete(memory_id)
            if memory_id in self._memory_ids:
                del self._memory_ids[memory_id]
            return True
        except Exception as e:
            print(f"Error in deleted method in LangMemLayer: \n\t{e.__class__.__name__}: {e}")
            return False

    def update(self, memory_id: str, **kwargs) -> bool:
        """Update the memory."""
        if "content" not in kwargs:
            raise KeyError("`content` is required in `kwargs`.")
        content = kwargs.pop("content")
        try:
            self.memory_layer.put(
                memory_id, 
                {"content": content}, 
                **kwargs
            )
            return True
        except Exception as e:
            print(f"Error in update method in LangMemLayer: \n\t{e.__class__.__name__}: {e}")
            return False

    def load_memory(self, user_id: Optional[str] = None) -> bool:
        """Load the memory of the user.""" 
        if user_id is None:
            user_id = self.config.user_id
        pkl_path = os.path.join(self.config.save_dir, f"{user_id}.pkl")
        config_path = os.path.join(self.config.save_dir, "config.json")
        if not os.path.exists(pkl_path) or not os.path.exists(config_path):
            return False 
        
        with open(config_path, 'r', encoding="utf-8") as f:
            config_dict = json.load(f)
        if user_id != config_dict["user_id"]:
            raise ValueError(
                f"The user id in the config file ({config_dict['user_id']}) "
                f"does not match the user id ({user_id}) in the function call."
            )
        config = LangMemConfig(**config_dict)
        self._llm_model = init_chat_model(config.llm_model)
        self._query_model = (
            None
            if config.query_model is None
            else init_chat_model(config.query_model)
        )
        self._store = InMemoryStore(
            index={
                "dims": config.retriever_dim, 
                "embed": config.retriever_name_or_path, 
                "fields": ["content"],   
            }
        )
        self.memory_layer = create_memory_store_manager(
            self._llm_model,
            enable_inserts=config.enable_inserts, 
            enable_deletes=config.enable_deletes, 
            query_model=self._query_model,  
            query_limit=config.query_limit, 
            namespace=("memories", config.user_id),  
            store=self._store, 
        )
        self.config = config 
        
        with open(pkl_path, "rb") as f:
            predefined_memory_units = pickle.load(f)
        self._memory_ids.clear()   

        for memory_unit in predefined_memory_units:
            self.memory_layer.put(**memory_unit) 
            self._memory_ids[memory_unit["key"]] = memory_unit["value"]

        return True 

    def save_memory(self) -> None:
        """Save the memory to a directory with config.json and memory .pkl."""
        os.makedirs(self.config.save_dir, exist_ok=True)

        # Write config.json
        config_path = os.path.join(self.config.save_dir, "config.json")
        config_dict = {
            "layer_type": self.layer_type,
            **self.config.model_dump()
        }
        with open(config_path, 'w', encoding="utf-8") as f:
            json.dump(config_dict, f, indent=4)

        # In LangMem, we don't store the vector embeddings. 
        preserved_memory_units = [] 
        for key, value in self._memory_ids.items(): 
            # Note that some memory units have been deleted. 
            if self.memory_layer.get(key) is not None:
                memory_unit = {
                    "key": key,
                    "value": value,
                }
                preserved_memory_units.append(memory_unit)

        pkl_path = os.path.join(self.config.save_dir, f"{self.config.user_id}.pkl")
        with open(pkl_path, "wb") as f:
            pickle.dump(preserved_memory_units, f)

In [8]:
import os 
os.environ["OPENAI_API_KEY"] = "sk-Y0bpFbtmUr1P2O33587a4a629aDf4aFeB50165C6BcA8A34c"
os.environ["OPENAI_API_BASE"] = "https://api.gptplus5.com/v1"

In [9]:
config = LangMemConfig(
    user_id="user_LongMemEval_longmemeval_0a995998", 
    retriever_name_or_path="huggingface:/mnt/dengxinle/pretrained_models/all-MiniLM-L6-v2",
    query_model="openai:gpt-4o-mini", 
    save_dir="LangMem/user_LongMemEval_longmemeval_0a995998"
) 
layer = LangMemLayer(config) 

In [10]:
layer.load_memory()

True

In [12]:
for key, value in layer._memory_ids.items(): 
    print(value)


{'kind': 'Memory', 'content': {'content': 'User continues to focus on cooking with seasonal ingredients, particularly citrus fruits like oranges and lemons, and is seeking inspiration for recipes that utilize these ingredients.'}}
{'kind': 'Memory', 'content': {'content': "User is interested in attending a classical concert at the Hollywood Bowl, specifically a performance by the Los Angeles Philharmonic of Tchaikovsky's 1812 Overture, which is typically performed during summer season or Independence Day celebrations, often in July or early August. Additionally, User is advised to check the Hollywood Bowl's official website for the most up-to-date schedule and ticket information and to consider signing up for the email newsletter or following the LA Philharmonic on social media for announcements."}}
{'kind': 'Memory', 'content': {'content': 'User is involved with a hedge fund that utilizes advanced scientific research and techniques for strategic decision-making in investments. The inv

In [4]:
from memories.datasets.longmemeval import LongMemEval
dataset = LongMemEval.read_raw_data(
    "/mnt/dengxinle/raspberry/memory_benchmark/longmemeval/longmemeval_s.json"
)

In [6]:
getattr(layer.llm_model, "generate")

<bound method BaseChatModel.generate of ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7252ebb8ed80>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7252eb3362d0>, root_client=<openai.OpenAI object at 0x7252fc7a38f0>, root_async_client=<openai.AsyncOpenAI object at 0x7252ebb8ee40>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.gptplus5.com/v1')>

In [5]:
?layer.llm_model.generate

Signature:
layer.llm_model.generate(
    messages: 'list[list[BaseMessage]]',
    stop: 'Optional[list[str]]' = None,
    callbacks: 'Callbacks' = None,
    *,
    tags: 'Optional[list[str]]' = None,
    metadata: 'Optional[dict[str, Any]]' = None,
    run_name: 'Optional[str]' = None,
    run_id: 'Optional[uuid.UUID]' = None,
    **kwargs: 'Any',
) -> 'LLMResult'
Docstring:
Pass a sequence of prompts to the model and return model generations.

This method should make use of batched calls for models that expose a batched
API.

Use this method when you want to:
    1. take advantage of batched calls,
    2. need more output from the model than just the top generated value,
    3. are building chains that are agnostic to the underlying language model
        type (e.g., pure text completion models vs chat models).

Args:
    messages: List of list of messages.
    stop: Stop words to use when generating. Model output is cut off at the
        first occurrence of any of these substrings.


In [5]:
dataset = dataset.sample(1, seed=42)
for trajectory, _ in dataset:
    session = trajectory[0]
    for message in session:
        layer.add_message(
            {"role": message.role, "content": message.content}, 
        )

[[HumanMessage(content="Use parallel tool calling to search for distinct memories relevant to this conversation.:\n\n<convo>\n================================ Human Message =================================\n\nI just got a new smart TV and I'm trying to figure out how to stream certain apps from my laptop to the TV. Do you know how I can do that? By the way, I've been noticing that my internet speeds have been slower than usual, especially during peak hours. I think it might be due to all the devices connected to our network.\n</convo>.", additional_kwargs={}, response_metadata={})]] 
-------
 
-------
 None 
-------
 None 
-------
 [] 
-------
 {} 
-------
 None 
-------
 None 
-------
 {'tools': [{'type': 'function', 'function': {'name': 'search_memory', 'description': 'Search your long-term memories for information relevant to your current context. Queries should be formatted as hypothetical memories that would be relevant to the current conversation.', 'parameters': {'properties': 

In [7]:
scores = [
    doc["metadata"]["score"]
    for doc in layer.retrieve(trajectory[1][0].content)
] 

In [9]:
layer.save_memory()

In [10]:
layer.load_memory()

True

In [11]:
true_scores = [
    doc["metadata"]["score"]
    for doc in layer.retrieve(trajectory[1][0].content)
] 

In [12]:
true_scores == scores

True

In [34]:
layer._memory_ids

{'55b0279b-5663-42d4-a162-afbd1f5ac305': {'kind': 'Memory',
  'content': {'content': 'User has a new smart TV and is trying to figure out how to stream certain apps from their laptop to the TV using Chromecast, which they think sounds the easiest. Additionally, they want to check their internet speed first and have asked for guidance on checking bandwidth usage and identifying devices that consume the most bandwidth. User has learned about Quality of Service (QoS) and that it can help prioritize traffic for their smart TV and other critical devices. They also understand how to check if their router supports QoS.'}},
 '22e25626-51f9-4d77-866e-a4c2ce845235': {'kind': 'Memory',
  'content': {'content': 'User is checking their router settings for a bandwidth monitoring feature and is open to trying third-party apps if it is unavailable. They also want to understand the differences between dual-band and tri-band routers and are questioning whether the extra cost of a tri-band router is just

In [51]:
for memory_id in layer._memory_ids: 
    if layer.memory_layer.get(memory_id):
        vector = layer._store._vectors[('memories', 'guest')][memory_id]
        memory_unit = {
            "key": memory_id,
            "value": layer._memory_ids[memory_id] 
        }
        layer.memory_layer.put(**memory_unit)
        new_vector = layer._store._vectors[('memories', 'guest')][memory_id]
        print(vector == new_vector)

True
True
True
True
True
True
True


In [41]:
print(len(layer._store._data[('memories', 'guest')]))

10


In [38]:
list(layer._store._data.keys())

[('memories', 'guest')]

In [41]:
layer._store.index_config

{'dims': 384,
 'embed': 'huggingface:/mnt/dengxinle/pretrained_models/all-MiniLM-L6-v2',
 '__tokenized_fields': [('$', '$')]}

In [35]:
for message in session: 
    print(f"{message.role}: {message.content}")
    print("-" * 50)

user: I just got a new smart TV and I'm trying to figure out how to stream certain apps from my laptop to the TV. Do you know how I can do that? By the way, I've been noticing that my internet speeds have been slower than usual, especially during peak hours. I think it might be due to all the devices connected to our network.
--------------------------------------------------
assistant: Congratulations on your new smart TV! Streaming content from your laptop to your TV is a great way to enhance your viewing experience. I'd be happy to help you with that.

There are a few ways to stream content from your laptop to your smart TV, depending on the devices and operating systems you're using. Here are a few common methods:

1. **Chromecast**: If your laptop has Google Chrome installed, you can use the Chromecast extension to cast content from your laptop to your TV. Make sure your Chromecast device is connected to the same WiFi network as your laptop.
2. **Miracast**: If your laptop and TV 

In [18]:
store = layer._store